# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,okha,53.5739,142.9478,15.93,88,100,3.76,RU,1724261344
1,1,invercargill,-46.4000,168.3500,3.34,92,42,0.45,NZ,1724261345
2,2,termas de rio hondo,-27.4998,-64.8604,14.25,74,100,2.71,AR,1724261346
3,3,yellowknife,62.4560,-114.3525,11.90,83,100,8.75,CA,1724261347
4,4,isafjordur,66.0755,-23.1240,7.02,81,75,8.75,IS,1724261348


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 700,
    frame_height=500,
    size='Humidity',
    color='City'
)

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
desired_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= 21.5) & (city_data_df['Humidity'] <= 50)]


# Drop any rows with null values
desired_cities_df.dropna(how='any')

# Display sample data
desired_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,happy valley-goose bay,53.3168,-60.3315,20.17,40,20,3.60,CA,1724261353
51,51,kapuskasing,49.4169,-82.4331,20.05,49,40,2.06,CA,1724261406
58,58,stuttgart,48.7823,9.1770,21.01,49,0,5.14,DE,1724260946
104,104,chibougamau,49.9168,-74.3659,19.79,40,20,4.12,CA,1724261469
115,115,cadillac,44.2519,-85.4012,19.95,49,0,3.09,US,1724261324
126,126,colonia,50.9333,6.9500,21.18,50,0,5.14,DE,1724261386
203,203,kolonia,50.9333,6.9500,21.18,50,0,5.14,DE,1724261386
204,204,alice springs,-23.7000,133.8833,15.75,44,1,2.57,AU,1724261590
205,205,khovd,48.0056,91.6419,19.73,27,66,3.05,MN,1724261591
212,212,elliot,-31.3333,27.8500,14.12,27,1,3.00,ZA,1724261601


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = desired_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,happy valley-goose bay,CA,53.3168,-60.3315,40,
51,kapuskasing,CA,49.4169,-82.4331,49,
58,stuttgart,DE,48.7823,9.1770,49,
104,chibougamau,CA,49.9168,-74.3659,40,
115,cadillac,US,44.2519,-85.4012,49,
126,colonia,DE,50.9333,6.9500,50,
203,kolonia,DE,50.9333,6.9500,50,
204,alice springs,AU,-23.7000,133.8833,44,
205,khovd,MN,48.0056,91.6419,27,
212,elliot,ZA,-31.3333,27.8500,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
   'apiKey' : geoapify_key,
   'format':'json',
   'categories':'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
happy valley-goose bay - nearest hotel: Hotel North Two
kapuskasing - nearest hotel: Travelodge
stuttgart - nearest hotel: Hotel Unger
chibougamau - nearest hotel: No hotel found
cadillac - nearest hotel: Holiday Inn
colonia - nearest hotel: Wasserturm Hotel Cologne
kolonia - nearest hotel: Wasserturm Hotel Cologne
alice springs - nearest hotel: Aurora Alice Springs
khovd - nearest hotel: Khovd hotel
elliot - nearest hotel: No hotel found
kasane - nearest hotel: Kasane Travel Lodge
naryn - nearest hotel: Ала-Тоо
thabazimbi - nearest hotel: Maroela Guest Lodge
senanga - nearest hotel: Senanga Safari Lodge
atkarsk - nearest hotel: No hotel found
lichtenburg - nearest hotel: Scott's Manor
okahandja - nearest hotel: Okahandja Country Hotel
harare - nearest hotel: No hotel found
mwinilunga - nearest hotel: No hotel found
mokopane - nearest hotel: Oasis Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
8,happy valley-goose bay,CA,53.3168,-60.3315,40,Hotel North Two
51,kapuskasing,CA,49.4169,-82.4331,49,Travelodge
58,stuttgart,DE,48.7823,9.1770,49,Hotel Unger
104,chibougamau,CA,49.9168,-74.3659,40,No hotel found
115,cadillac,US,44.2519,-85.4012,49,Holiday Inn
126,colonia,DE,50.9333,6.9500,50,Wasserturm Hotel Cologne
203,kolonia,DE,50.9333,6.9500,50,Wasserturm Hotel Cologne
204,alice springs,AU,-23.7000,133.8833,44,Aurora Alice Springs
205,khovd,MN,48.0056,91.6419,27,Khovd hotel
212,elliot,ZA,-31.3333,27.8500,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='EsriTerrain',
    coastline=True,
    hover_cols=['Hotel Name','Country'],
    color='City',
    size='Humidity',
    frame_width=900,
    frame_height=500
)

# Display the map
map_plot2

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)
   .Coastline.I :Feature   [Longitude,Latitude]